In [18]:
import pandas as pd
import re
import string

In [19]:
df_false = pd.read_csv('df_Fake_Arxiv.csv')
df_true = pd.read_csv('df_True_CBS.csv')

In [20]:
df_false.drop('Unnamed: 0', axis=1, inplace=True)
df_true.drop('Unnamed: 0', axis=1, inplace=True)

In [21]:
df_true['class'] = 1

In [22]:
df_false[['text', 'class']] = df_false[['content', 'label']]

In [23]:
df_false.drop(['content', 'label'], axis=1, inplace=True)
df_true.drop('Link', axis=1, inplace=True)

In [24]:
df_false

,text,class
0,Source: Angela George/CC BY-SA 3.0\n\nOne of m...,0
1,"""He gave them details of what may have occurre...",0
2,Like their respectively memorable characters i...,0
3,A cursory glance at the Spring ’16 collections...,0
4,Liza Weil and Paul Adelstein’s divorce has bee...,0
...,...,...
23132,Yeah. I was definitely seeing this. A substant...,0
23133,The World Health Organization (WHO) is working...,0
23134,"""We must all learn to control and manage this ...",0
23135,What’s the relationship between low levels of ...,0


In [25]:
df_true

,text,class
0,Beijing — A senior Chinese arms control offic...,1
1,The wreckage of an Italian submarine that was...,1
2,Beijing — China wants to use the Beijing Wint...,1
3,London — It's a critical week for Britain's P...,1
4,Brazilian President Jair Bolsonaro was rushed...,1
...,...,...
4792,Wildfires burning across Australia's two most...,1
4793,"Former Nissan CEO Carlos Ghosn, who is awaiti...",1
4794,Northern England — As the civil war in Syria ...,1
4795,Beijing — A Chinese court on Monday sentenced...,1


In [ ]:
def remove_space(df, column, space):
    df[column] = df[column].apply(lambda x: x.replace(space, ' '))
    df[column] = df[column].apply(lambda x: x[1:] if x.startswith(' ') else x)

## REMOVE DUPLICATES
print('Size of df_true: ', df_true.shape)
print('Size of df_false: ', df_false.shape)
df_true.drop_duplicates(subset=['text'], inplace=True)
df_false.drop_duplicates(subset=['text'], inplace=True)
print('New Size of df_true: ', df_true.shape)
print('New Size of df_false: ', df_false.shape)
df_false = df_false.reset_index(drop=True)
df_true = df_true.reset_index(drop=True)

##DATA FOR MANUAL TESTING
manual_fake = df_false.tail(10)
for i in range(17448, 17438, -1):
    df_false.drop([i], axis=0, inplace=True)
    
manual_true = df_true.tail(10)
for i in range(21191, 21181, -1):
    df_true.drop([i], axis=0, inplace=True)
    
manual = pd.concat([manual_fake, manual_true], axis = 0)

##CONCATING THE TRUE AND FALSE DATASETS
df = pd.concat([df_true, df_false], axis=0)

##REMOVING THE NULL VALUES AND SPACES
null_values = df[df['text'] == ' '].index
df.drop(null_values, axis=0, inplace=True)

null_values = df[df['title'] == ' '].index
df.drop(null_values, axis=0, inplace=True)

##REMOVING THE SPECIAL CHARACTERS
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', ' ', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

df['title'] = df['title'].apply(word_drop)
df['text'] = df['text'].apply(word_drop)
manual['title'] = manual['title'].apply(word_drop)
manual['text'] = manual['text'].apply(word_drop)

remove_space(df, 'title', '    ')
remove_space(df, 'title', '   ')
remove_space(df, 'title', '  ')
remove_space(df, 'text', '    ')
remove_space(df, 'text', '   ')
remove_space(df, 'text', '  ')
remove_space(manual, 'title', '    ')
remove_space(manual, 'title', '   ')
remove_space(manual, 'title', '  ')
remove_space(manual, 'text', '    ')
remove_space(manual, 'text', '   ')
remove_space(manual, 'text', '  ')

df['text'] = df['text'].apply(lambda x: x.replace('u s', 'usa'))
df['title'] = df['title'].apply(lambda x: x.replace('u s', 'usa'))
manual['text'] = manual['text'].apply(lambda x: x.replace('u s', 'usa'))
manual['title'] = manual['title'].apply(lambda x: x.replace('u s', 'usa'))
df['text'] = df['text'].apply(lambda x: x.replace('u s a', 'usa'))
df['title'] = df['title'].apply(lambda x: x.replace('u s a', 'usa'))
manual['text'] = manual['text'].apply(lambda x: x.replace('u s a', 'usa'))
manual['title'] = manual['title'].apply(lambda x: x.replace('u s a', 'usa'))

print(df.isnull().sum())
df.dropna()
print(df.isnull().sum())

##RE-INDEXING
df = df.reset_index(drop=True)
manual = manual.reset_index(drop=True)

##CHECKING THE FINAL SIZE
print('Last Size of df_true: ', df_true.shape)
print('Last Size of df_false: ', df_false.shape)
print('Last Size of df: ', df.shape)

##EXPORTING THE 'CLEANED' DATA
df.to_csv('dataset.csv')
manual.to_csv('manual_testing.csv')